# Pré-Requisitos

## Sweetviz - https://pypi.org/project/sweetviz/

In [ ]:
!pip install sweetviz

## Baixa e Descompacta o Dataset

In [ ]:
url = "http://ec2-100-25-20-177.compute-1.amazonaws.com/coletas/"
# JSON (3 GB):
#filename = "prefeito_rio_2020.json.tar.gz"
# CSV (1 GB):
filename = "prefeito_rio_2020.csv.tar.gz"
!wget {url + filename}
!tar -xvzf $filename

### Total de linhas (registros) no arquivo JSON

In [ ]:
count = 0

for line in open("prefeito_rio_2020.json"):
  count += 1

print(count)

# Bibliotecas

In [ ]:
import pandas as pd
import sweetviz as sv

# Abre o DataSet diretamente do GitHub

In [ ]:
uri = "prefeito_rio_2020.csv"
ds = pd.read_csv(uri)
ds.head()

# Mostra informações sobre o DataSet

In [ ]:
ds.info()

# Seleciona registros com dados nulos

In [ ]:
ds[ds.isna().any(axis=1)]

# Mostra total de valores nulos por colunas

In [ ]:
ds.isna().sum()

# Mostra as colunas informando sobre valores nulos

In [ ]:
ds.isna().any()

# Lista somente as colunas que possuem valores nulos

In [ ]:
ds.columns[ds.isna().any()].tolist()

# Análise Exploratória de Dados - Sweetviz

In [ ]:
report = sv.analyze(ds)
report.show_notebook()

## Exporta relatório para HTML

In [ ]:
report.show_html()